**DeepSort**

In [1]:
#Install dependencies
!pip install ultralytics deep_sort_realtime opencv-python --quiet

#Import libraries
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from google.colab import files
import os
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# Paths (use your uploaded files)
video_file = "15sec_input_720p.mp4"
model_file = "best.pt"

# Load model
model = YOLO(model_file)

# Open video
cap = cv2.VideoCapture(video_file)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output
os.makedirs("output", exist_ok=True)
output_path = "output/tracked_output_with_ball.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Tracker
tracker = DeepSort(max_age=30)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []
    confs = results[0].boxes.conf.cpu().numpy() if results[0].boxes else []
    class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes else []

    detections = []
    labels = []

    for box, conf, cls_id in zip(boxes, confs, class_ids):
        if int(cls_id) in [0, 1] and conf > 0.3:  # class 0 = player, class 1 = ball
            x1, y1, x2, y2 = map(int, box)
            label = "player" if int(cls_id) == 0 else "ball"
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, label))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, w, h = track.to_ltrb()
        label = track.get_det_class()  # returns 'player' or 'ball'

        color = (0, 255, 0) if label == 'player' else (0, 165, 255)  # green for player, orange for ball
        tag = f"{label.capitalize()} {track_id}"

        cv2.rectangle(frame, (int(l), int(t)), (int(l + w), int(t + h)), color, 2)
        cv2.putText(frame, tag, (int(l), int(t) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)

cap.release()
out.release()
print("Output saved at", output_path)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 

In [2]:
from google.colab import files
files.download("output/tracked_output_with_ball.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Different Video(DeepSort)**

In [3]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# File paths
video_file = "/content/6077718-uhd_3840_2160_25fps.mp4"
model_file = "best.pt"

# Load YOLOv11 model
model = YOLO(model_file)

# Open video
cap = cv2.VideoCapture(video_file)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output video setup
os.makedirs("output", exist_ok=True)
output_path = "output/tracked_shrunk_boxes.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Initialize tracker
tracker = DeepSort(max_age=30)
frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []
    confs = results[0].boxes.conf.cpu().numpy() if results[0].boxes else []
    class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes else []

    detections = []

    for box, conf, cls_id in zip(boxes, confs, class_ids):
        if int(cls_id) in [0, 1] and conf > 0.2:
            x1, y1, x2, y2 = map(int, box)

            #Shrink box size by 10%
            w = x2 - x1
            h = y2 - y1
            shrink = 0.1  # 10%

            x1_new = int(x1 + w * shrink / 2)
            y1_new = int(y1 + h * shrink / 2)
            x2_new = int(x2 - w * shrink / 2)
            y2_new = int(y2 - h * shrink / 2)

            label = "player" if int(cls_id) == 0 else "ball"
            detections.append(([x1_new, y1_new, x2_new - x1_new, y2_new - y1_new], conf, label))

    # Print detections per frame
    print(f"[Frame {frame_id}] Detections: {len(detections)}")

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, w, h = track.to_ltrb()
        label = track.get_det_class()
        color = (0, 255, 0) if label == 'player' else (0, 165, 255)
        tag = f"{label.capitalize()} {track_id}"

        cv2.rectangle(frame, (int(l), int(t)), (int(l + w), int(t + h)), color, 2)
        cv2.putText(frame, tag, (int(l), int(t) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)

cap.release()
out.release()
print("Output saved at", output_path)



0: 384x640 6 players, 2 referees, 56.2ms
Speed: 3.4ms preprocess, 56.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
[Frame 1] Detections: 0

0: 384x640 5 players, 2 referees, 59.1ms
Speed: 16.4ms preprocess, 59.1ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
[Frame 2] Detections: 0

0: 384x640 6 players, 2 referees, 47.2ms
Speed: 4.5ms preprocess, 47.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
[Frame 3] Detections: 0

0: 384x640 7 players, 2 referees, 50.4ms
Speed: 8.2ms preprocess, 50.4ms inference, 4.7ms postprocess per image at shape (1, 3, 384, 640)
[Frame 4] Detections: 0

0: 384x640 9 players, 3 referees, 55.0ms
Speed: 3.8ms preprocess, 55.0ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
[Frame 5] Detections: 0

0: 384x640 6 players, 2 referees, 81.7ms
Speed: 5.6ms preprocess, 81.7ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)
[Frame 6] Detections: 0

0: 384x640 9 players

In [4]:
from google.colab import files
files.download("output/tracked_shrunk_boxes.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# File paths
video_file = "15sec_input_720p.mp4"
model_file = "best.pt"

# Load YOLO model
model = YOLO(model_file)

# Video input/output
cap = cv2.VideoCapture(video_file)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

os.makedirs("output", exist_ok=True)
output_path = "output/tracked_players_only.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Tracker
tracker = DeepSort(max_age=30)
frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []
    confs = results[0].boxes.conf.cpu().numpy() if results[0].boxes else []
    class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes else []

    detections = []

    for box, conf, cls_id in zip(boxes, confs, class_ids):
        if int(cls_id) == 0 and conf > 0.2:  #Only players
            x1, y1, x2, y2 = map(int, box)

            # Shrink box by 10%
            w = x2 - x1
            h = y2 - y1
            shrink = 0.1

            x1_new = int(x1 + w * shrink / 2)
            y1_new = int(y1 + h * shrink / 2)
            x2_new = int(x2 - w * shrink / 2)
            y2_new = int(y2 - h * shrink / 2)

            label = "player"
            detections.append(([x1_new, y1_new, x2_new - x1_new, y2_new - y1_new], conf, label))

    print(f"[Frame {frame_id}] Player Detections: {len(detections)}")

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, w, h = track.to_ltrb()
        label = "player"
        color = (0, 255, 0)  # green

        cv2.rectangle(frame, (int(l), int(t)), (int(l + w), int(t + h)), color, 2)
        cv2.putText(frame, f"Player {track_id}", (int(l), int(t) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)

cap.release()
out.release()
print("Player-only output saved at", output_path)



0: 384x640 1 ball, 16 players, 2 referees, 46.6ms
Speed: 2.0ms preprocess, 46.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[Frame 1] Player Detections: 1

0: 384x640 18 players, 2 referees, 35.0ms
Speed: 2.1ms preprocess, 35.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
[Frame 2] Player Detections: 0

0: 384x640 1 ball, 16 players, 2 referees, 33.9ms
Speed: 2.2ms preprocess, 33.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[Frame 3] Player Detections: 1

0: 384x640 1 ball, 14 players, 2 referees, 33.9ms
Speed: 2.1ms preprocess, 33.9ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
[Frame 4] Player Detections: 1

0: 384x640 1 ball, 14 players, 2 referees, 32.2ms
Speed: 12.7ms preprocess, 32.2ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
[Frame 5] Player Detections: 1

0: 384x640 1 ball, 16 players, 2 referees, 31.6ms
Speed: 2.2ms preprocess, 31.6ms inference, 2.1ms postprocess 

In [6]:
from google.colab import files
files.download("output/tracked_players_only.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**ShrinkBox20**

In [7]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# File paths
video_file = "15sec_input_720p.mp4"
model_file = "best.pt"

# Load YOLO model
model = YOLO(model_file)

# Video input/output
cap = cv2.VideoCapture(video_file)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

os.makedirs("output", exist_ok=True)
output_path = "output/tracked_players_shrink20.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Tracker
tracker = DeepSort(max_age=30)
frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []
    confs = results[0].boxes.conf.cpu().numpy() if results[0].boxes else []
    class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes else []

    detections = []

    for box, conf, cls_id in zip(boxes, confs, class_ids):
        if int(cls_id) == 0 and conf > 0.2:
            x1, y1, x2, y2 = map(int, box)

            # Shrink box by 20%
            w = x2 - x1
            h = y2 - y1
            shrink = 0.2  # 20%

            x1_new = int(x1 + w * shrink / 2)
            y1_new = int(y1 + h * shrink / 2)
            x2_new = int(x2 - w * shrink / 2)
            y2_new = int(y2 - h * shrink / 2)

            label = "player"
            detections.append(([x1_new, y1_new, x2_new - x1_new, y2_new - y1_new], conf, label))

    print(f"[Frame {frame_id}] Player Detections: {len(detections)}")

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, w, h = track.to_ltrb()
        color = (0, 255, 0)
        cv2.rectangle(frame, (int(l), int(t)), (int(l + w), int(t + h)), color, 2)
        cv2.putText(frame, f"Player {track_id}", (int(l), int(t) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)

cap.release()
out.release()
print("20% shrinked player-only output saved at", output_path)



0: 384x640 1 ball, 16 players, 2 referees, 53.7ms
Speed: 2.4ms preprocess, 53.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
[Frame 1] Player Detections: 1

0: 384x640 18 players, 2 referees, 49.8ms
Speed: 3.6ms preprocess, 49.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
[Frame 2] Player Detections: 0

0: 384x640 1 ball, 16 players, 2 referees, 44.9ms
Speed: 2.0ms preprocess, 44.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
[Frame 3] Player Detections: 1

0: 384x640 1 ball, 14 players, 2 referees, 35.9ms
Speed: 2.1ms preprocess, 35.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
[Frame 4] Player Detections: 1

0: 384x640 1 ball, 14 players, 2 referees, 36.0ms
Speed: 2.1ms preprocess, 36.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[Frame 5] Player Detections: 1

0: 384x640 1 ball, 16 players, 2 referees, 33.6ms
Speed: 4.4ms preprocess, 33.6ms inference, 2.1ms postprocess p

In [8]:
from google.colab import files
files.download("output/tracked_players_shrink20.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Shrink30**

In [9]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# File paths
video_file = "15sec_input_720p.mp4"
model_file = "best.pt"

# Load YOLO model
model = YOLO(model_file)

# Video input/output
cap = cv2.VideoCapture(video_file)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

os.makedirs("output", exist_ok=True)
output_path = "output/tracked_players_shrink30.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Tracker
tracker = DeepSort(max_age=30)
frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []
    confs = results[0].boxes.conf.cpu().numpy() if results[0].boxes else []
    class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes else []

    detections = []

    for box, conf, cls_id in zip(boxes, confs, class_ids):
        if int(cls_id) == 0 and conf > 0.2:
            x1, y1, x2, y2 = map(int, box)

            # Shrink box by 30%
            w = x2 - x1
            h = y2 - y1
            shrink = 0.3  # 30%

            x1_new = int(x1 + w * shrink / 2)
            y1_new = int(y1 + h * shrink / 2)
            x2_new = int(x2 - w * shrink / 2)
            y2_new = int(y2 - h * shrink / 2)

            detections.append(([x1_new, y1_new, x2_new - x1_new, y2_new - y1_new], conf, "player"))

    print(f"[Frame {frame_id}] Player Detections: {len(detections)}")

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, w, h = track.to_ltrb()
        color = (0, 255, 0)
        cv2.rectangle(frame, (int(l), int(t)), (int(l + w), int(t + h)), color, 2)
        cv2.putText(frame, f"Player {track_id}", (int(l), int(t) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)

cap.release()
out.release()
print("30% shrinked player-only output saved at", output_path)



0: 384x640 1 ball, 16 players, 2 referees, 43.4ms
Speed: 2.4ms preprocess, 43.4ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
[Frame 1] Player Detections: 1

0: 384x640 18 players, 2 referees, 33.5ms
Speed: 2.4ms preprocess, 33.5ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
[Frame 2] Player Detections: 0

0: 384x640 1 ball, 16 players, 2 referees, 34.7ms
Speed: 2.2ms preprocess, 34.7ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
[Frame 3] Player Detections: 1

0: 384x640 1 ball, 14 players, 2 referees, 33.0ms
Speed: 2.2ms preprocess, 33.0ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)
[Frame 4] Player Detections: 1

0: 384x640 1 ball, 14 players, 2 referees, 32.9ms
Speed: 2.1ms preprocess, 32.9ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
[Frame 5] Player Detections: 1

0: 384x640 1 ball, 16 players, 2 referees, 33.0ms
Speed: 2.0ms preprocess, 33.0ms inference, 1.8ms postprocess p

In [10]:
from google.colab import files
files.download("output/tracked_players_shrink30.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**ReID with DeepSort**

In [11]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# Paths
video_file = "15sec_input_720p.mp4"
model_file = "best.pt"

# Load model
model = YOLO(model_file)

# Video setup
cap = cv2.VideoCapture(video_file)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output setup
os.makedirs("output", exist_ok=True)
output_path = "output/tracked_players_reid_fixed.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

#DeepSORT tracker tuned for better Re-ID
tracker = DeepSort(
    max_age=80,              # Keep track alive longer (even if player disappears)
    n_init=1,                # Confirm ID faster
    max_cosine_distance=0.2  # Stricter visual match
)

frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []
    confs = results[0].boxes.conf.cpu().numpy() if results[0].boxes else []
    class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes else []

    detections = []

    for box, conf, cls_id in zip(boxes, confs, class_ids):
        if int(cls_id) == 0 and conf > 0.2:  #Only players
            x1, y1, x2, y2 = map(int, box)
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, "player"))

    print(f"[Frame {frame_id}] Players: {len(detections)}")

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, w, h = track.to_ltrb()
        label = track.get_det_class()

        color = (0, 255, 0)
        tag = f"{label.capitalize()} {track_id}"

        cv2.rectangle(frame, (int(l), int(t)), (int(l + w), int(t + h)), color, 2)
        cv2.putText(frame, tag, (int(l), int(t) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)

cap.release()
out.release()
print("ReID-fixed output saved at", output_path)



0: 384x640 1 ball, 16 players, 2 referees, 46.8ms
Speed: 2.7ms preprocess, 46.8ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
[Frame 1] Players: 1

0: 384x640 18 players, 2 referees, 34.2ms
Speed: 2.1ms preprocess, 34.2ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
[Frame 2] Players: 0

0: 384x640 1 ball, 16 players, 2 referees, 36.8ms
Speed: 2.2ms preprocess, 36.8ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
[Frame 3] Players: 1

0: 384x640 1 ball, 14 players, 2 referees, 33.8ms
Speed: 2.2ms preprocess, 33.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[Frame 4] Players: 1

0: 384x640 1 ball, 14 players, 2 referees, 34.5ms
Speed: 2.2ms preprocess, 34.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
[Frame 5] Players: 1

0: 384x640 1 ball, 16 players, 2 referees, 34.8ms
Speed: 2.1ms preprocess, 34.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
[Frame 6] Playe

In [12]:
from google.colab import files
files.download("output/tracked_players_reid_fixed.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# Paths
video_file = "15sec_input_720p.mp4"
model_file = "best.pt"

# Load model
model = YOLO(model_file)

# Video setup
cap = cv2.VideoCapture(video_file)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output setup
os.makedirs("output", exist_ok=True)
output_path = "output/tracked_players_reid_cleaned.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

#DeepSORT tracker tuned for better Re-ID
tracker = DeepSort(
    max_age=80,
    n_init=1,
    max_cosine_distance=0.2
)

frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []
    confs = results[0].boxes.conf.cpu().numpy() if results[0].boxes else []
    class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes else []

    detections = []

    for box, conf, cls_id in zip(boxes, confs, class_ids):
        if int(cls_id) == 0 and conf > 0.2:
            x1, y1, x2, y2 = map(int, box)
            detections.append(([x1, y1, x2 - x1, y2 - y1], conf, "player"))

    print(f"[Frame {frame_id}] Players: {len(detections)}")

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        #Skip ghost tracks with no recent update
        if track.time_since_update > 1:
            continue

        track_id = track.track_id
        l, t, w, h = track.to_ltrb()
        label = track.get_det_class()

        color = (0, 255, 0)
        tag = f"{label.capitalize()} {track_id}"

        cv2.rectangle(frame, (int(l), int(t)), (int(l + w), int(t + h)), color, 2)
        cv2.putText(frame, tag, (int(l), int(t) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    out.write(frame)

cap.release()
out.release()
print("Ghost-free output saved at", output_path)



0: 384x640 1 ball, 16 players, 2 referees, 44.5ms
Speed: 1.8ms preprocess, 44.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[Frame 1] Players: 1

0: 384x640 18 players, 2 referees, 35.7ms
Speed: 2.3ms preprocess, 35.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
[Frame 2] Players: 0

0: 384x640 1 ball, 16 players, 2 referees, 36.4ms
Speed: 2.2ms preprocess, 36.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
[Frame 3] Players: 1

0: 384x640 1 ball, 14 players, 2 referees, 35.4ms
Speed: 2.1ms preprocess, 35.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)
[Frame 4] Players: 1

0: 384x640 1 ball, 14 players, 2 referees, 34.7ms
Speed: 2.2ms preprocess, 34.7ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
[Frame 5] Players: 1

0: 384x640 1 ball, 16 players, 2 referees, 34.7ms
Speed: 2.2ms preprocess, 34.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
[Frame 6] Playe

In [14]:
from google.colab import files
files.download("output/tracked_players_reid_cleaned.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# File paths
video_file = "/content/6077718-uhd_3840_2160_25fps.mp4"
model_file = "best.pt"

# Load YOLOv11 model
model = YOLO(model_file)

# Video setup
cap = cv2.VideoCapture(video_file)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

os.makedirs("output", exist_ok=True)
output_path = "output/stable_tracked_players.mp4"
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

#DeepSORT tracker — tuned for stable IDs
tracker = DeepSort(
    max_age=40,               # Keep object alive longer
    n_init=2,                 # Confirm ID faster
    max_cosine_distance=0.3  # Allow visual variation
)

frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []
    confs = results[0].boxes.conf.cpu().numpy() if results[0].boxes else []
    class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes else []

    detections = []

    for box, conf, cls_id in zip(boxes, confs, class_ids):
        if int(cls_id) == 0 and conf > 0.15:  #Lower threshold for more players
            x1, y1, x2, y2 = map(int, box)

            #Shrink box only slightly (10%)
            w = x2 - x1
            h = y2 - y1
            shrink = 0.1

            x1_new = int(x1 + w * shrink / 2)
            y1_new = int(y1 + h * shrink / 2)
            x2_new = int(x2 - w * shrink / 2)
            y2_new = int(y2 - h * shrink / 2)

            detections.append(([x1_new, y1_new, x2_new - x1_new, y2_new - y1_new], conf, "player"))

    print(f"[Frame {frame_id}] Player Detections: {len(detections)}")

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        l, t, w, h = track.to_ltrb()
        cv2.rectangle(frame, (int(l), int(t)), (int(l + w), int(t + h)), (0, 255, 0), 2)
        cv2.putText(frame, f"Player {track_id}", (int(l), int(t) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print("DONE: Output saved at", output_path)



0: 384x640 6 players, 2 referees, 52.3ms
Speed: 3.3ms preprocess, 52.3ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
[Frame 1] Player Detections: 0

0: 384x640 5 players, 2 referees, 41.3ms
Speed: 4.0ms preprocess, 41.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
[Frame 2] Player Detections: 0

0: 384x640 6 players, 2 referees, 41.3ms
Speed: 3.4ms preprocess, 41.3ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)
[Frame 3] Player Detections: 0

0: 384x640 7 players, 2 referees, 41.4ms
Speed: 4.6ms preprocess, 41.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)
[Frame 4] Player Detections: 0

0: 384x640 9 players, 3 referees, 45.1ms
Speed: 3.4ms preprocess, 45.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
[Frame 5] Player Detections: 0

0: 384x640 6 players, 2 referees, 45.8ms
Speed: 3.6ms preprocess, 45.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[Frame 6] P

In [16]:
from google.colab import files
files.download("output/stable_tracked_players.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**ByteTrack(Better Result)**

In [17]:
!pip install ultralytics opencv-python --quiet

In [18]:
from ultralytics import YOLO
import os

# File paths
video_path = "15sec_input_720p.mp4"  # Upload your input video
model_path = "best.pt"               # Upload your trained YOLOv11 model
output_path = "output/bytetrack_output.mp4"

# Create output folder
os.makedirs("output", exist_ok=True)

# Load YOLO model
model = YOLO(model_path)

# Run ByteTrack tracking
results = model.track(
    source=video_path,
    tracker="bytetrack.yaml",  # Use ByteTrack
    persist=True,              # Stable IDs
    conf=0.2,                  # Lower threshold to catch more players
    save=True,                 # Save result to file
    show=False                 # Set True only for local system (not in Colab)
)


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 0.6s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/375) /content/15sec_input_720p.mp4: 384x640 1 ball, 16 players, 2 referees, 61.2ms
video 1/1 (frame 2/375) /content/15sec_input_720p.mp4: 384x640 15 players, 2 referees, 72.2ms
video 1/1 (frame 3/375) /content/15sec_inpu

In [19]:
import glob
import shutil
import os

output_path = "output/bytetrack_output.mp4"

# Detect latest ByteTrack folder
track_dirs = sorted(glob.glob("runs/detect/track*"), key=os.path.getmtime, reverse=True)

if track_dirs:
    latest_dir = track_dirs[0]
    print("Found tracking folder:", latest_dir)

    # Search for any common video format
    video_files = []
    for ext in ["*.mp4", "*.avi", "*.mov", "*.mkv"]:
        video_files.extend(glob.glob(os.path.join(latest_dir, ext)))

    if video_files:
        src_file = video_files[0]
        shutil.copy(src_file, output_path)
        print(f"Output video copied to {output_path}")
    else:
        print("No video file (.mp4/.avi/.mov) found in:", latest_dir)
else:
    print("Could not find any tracking output folder.")



Found tracking folder: runs/detect/track
Output video copied to output/bytetrack_output.mp4


In [20]:
from google.colab import files
files.download("output/bytetrack_output.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Norfair**

In [21]:
!pip install norfair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 55.1 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=c83da6e2094a5f346598c4e7963dab1f60f1fe0f1a269b62c1ae59f64b89b3da
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3

In [22]:
import cv2
from ultralytics import YOLO
from norfair import Detection, Tracker, Video
import numpy as np
import os

# Load YOLOv11 Model
model = YOLO("best.pt")
video_path = "15sec_input_720p.mp4"
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Output setup
os.makedirs("output", exist_ok=True)
out_path = "output/norfair_output.mp4"
out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Norfair Tracker (basic config)
tracker = Tracker(distance_function="euclidean", distance_threshold=30)

def yolo_detections_to_norfair(boxes, confs):
    detections = []
    for box, conf in zip(boxes, confs):
        x1, y1, x2, y2 = map(int, box)
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2
        detections.append(Detection(points=np.array([[x_center, y_center]]), scores=np.array([conf])))
    return detections

# Main loop
while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)
    boxes = results[0].boxes.xyxy.cpu().numpy() if results[0].boxes else []
    confs = results[0].boxes.conf.cpu().numpy() if results[0].boxes else []
    class_ids = results[0].boxes.cls.cpu().numpy() if results[0].boxes else []

    # Only keep class 0 (player)
    filtered_boxes = []
    filtered_confs = []
    for box, conf, cls_id in zip(boxes, confs, class_ids):
        if int(cls_id) == 0 and conf > 0.2:
            filtered_boxes.append(box)
            filtered_confs.append(conf)

    norfair_detections = yolo_detections_to_norfair(filtered_boxes, filtered_confs)
    tracked_objects = tracker.update(detections=norfair_detections)

    # Draw tracks
    for obj in tracked_objects:
        x, y = map(int, obj.estimate[0])
        cv2.circle(frame, (x, y), 5, (0, 255, 0), -1)
        cv2.putText(frame, f"Player {obj.id}", (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    out.write(frame)

cap.release()
out.release()
print(f"Norfair output saved to {out_path}")



0: 384x640 1 ball, 16 players, 2 referees, 53.6ms
Speed: 2.0ms preprocess, 53.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 47.9ms
Speed: 2.6ms preprocess, 47.9ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 47.9ms
Speed: 2.1ms preprocess, 47.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 37.8ms
Speed: 2.3ms preprocess, 37.8ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 35.0ms
Speed: 2.2ms preprocess, 35.0ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 38.5ms
Speed: 2.3ms preprocess, 38.5ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 34.3ms
Speed: 2.6ms preprocess, 34.3ms inference, 2.2ms postprocess per image at sha

In [23]:
from google.colab import files
files.download("output/norfair_output.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Modified_Bytetrack**

In [24]:
from ultralytics import YOLO
import os

video_path = "15sec_input_720p.mp4"
model_path = "best.pt"

# Clean any previous output
!rm -rf runs

# Load model
model = YOLO(model_path)

# Run ByteTrack Tracking
results = model.track(
    source=video_path,
    tracker="bytetrack.yaml",
    persist=True,
    conf=0.2,
    save=True,
    save_dir="runs/track",  # force path
    show=False,
    stream=False            # disable stream for safer write
)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/375) /content/15sec_input_720p.mp4: 384x640 1 ball, 16 players, 2 referees, 40.3ms
video 1/1 (frame 2/375) /content/15sec_input_720p.mp4: 384x640 15 players, 2 referees, 35.1ms
video 1/1 (frame 3/375) /content/15sec_input_720p.mp4: 384x640 16 players, 2 referees, 34.8ms
video 1/1 (frame 4/375) /content/15sec_input_720p.mp4: 384x640 1 ball, 14 players, 1 referee, 33.4ms
video 1/1 (frame 5/375) /content/15sec_input_720p.mp4: 384x640 1 

In [25]:
results = model.track(
    source=video_path,
    tracker="botsort_custom.yaml",
    persist=True,
    stream=True,
    conf=0.2,
    save=True,
    show=False
)

In [26]:
import os
print(os.listdir("runs/detect/track"))

['15sec_input_720p.avi']


In [27]:
import cv2

input_video = "runs/detect/track/15sec_input_720p.avi"
output_video = "runs/detect/track/15sec_input_720p_converted.mp4"

cap = cv2.VideoCapture(input_video)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    out.write(frame)

cap.release()
out.release()
print("Converted to MP4:", output_video)


Converted to MP4: runs/detect/track/15sec_input_720p_converted.mp4


In [28]:
from google.colab import files

files.download("runs/detect/track/15sec_input_720p_converted.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Botsort**

In [29]:
!pip install ultralytics opencv-python --quiet

In [30]:
botsort_config = """
tracker_type: botsort
track_high_thresh: 0.4
track_low_thresh: 0.2
new_track_thresh: 0.4
track_buffer: 60
match_thresh: 0.8
gmc_method: none
proximity_thresh: 0.5
appearance_thresh: 0.25
with_reid: False
fast_reid: config/fast_reid.yaml
device: cpu
"""
with open("botsort_custom.yaml", "w") as f:
    f.write(botsort_config)
print("BoT-SORT config file created.")


BoT-SORT config file created.


In [31]:
from ultralytics import YOLO
import os

video_path = "15sec_input_720p.mp4"
model_path = "best.pt"
output_dir = "runs/detect/track"
os.makedirs(output_dir, exist_ok=True)

model = YOLO(model_path)

results = model.track(
    source="15sec_input_720p.mp4",
    tracker="botsort.yaml",     # Use built-in safe config
    persist=True,
    stream=False,
    conf=0.2,
    save=True,
    show=False,
    verbose=True
)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/375) /content/15sec_input_720p.mp4: 384x640 1 ball, 16 players, 2 referees, 58.6ms
video 1/1 (frame 2/375) /content/15sec_input_720p.mp4: 384x640 16 players, 2 referees, 42.8ms
video 1/1 (frame 3/375) /content/15sec_input_720p.mp4: 384x640 15 players, 2 referees, 42.3ms
video 1/1 (frame 4/375) /content/15sec_input_720p.mp4: 384x640 14 players, 2 referees, 42.3ms
video 1/1 (frame 5/375) /content/15sec_input_720p.mp4: 384x640 14 player

In [32]:
import os
import cv2

# Search for AVI output
avi_files = [f for f in os.listdir(output_dir) if f.endswith(".avi")]
print(avi_files)

# Convert to MP4 if AVI exists
if avi_files:
    input_avi = os.path.join(output_dir, avi_files[0])
    output_mp4 = input_avi.replace(".avi", "_converted.mp4")

    cap = cv2.VideoCapture(input_avi)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(output_mp4, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()
    print("Converted to MP4:", output_mp4)
else:
    print("No AVI file found in output.")


['15sec_input_720p.avi']
Converted to MP4: runs/detect/track/15sec_input_720p_converted.mp4


In [33]:
from google.colab import files
files.download("runs/detect/track/15sec_input_720p_converted.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**CustomBotsort**

In [34]:
yaml_content = """
tracker_type: botsort
track_high_thresh: 0.6
track_low_thresh: 0.1
new_track_thresh: 0.7
track_buffer: 30
match_thresh: 0.8
gmc: False
orb_features: 1000
"""

with open("botsort_custom.yaml", "w") as f:
    f.write(yaml_content)

print("Fixed botsort_custom.yaml created (tracker_type lowercase).")




Fixed botsort_custom.yaml created (tracker_type lowercase).


In [35]:
from ultralytics import YOLO
import os

video_path = "15sec_input_720p.mp4"
model_path = "best.pt"
output_dir = "runs/detect/track"
os.makedirs(output_dir, exist_ok=True)

model = YOLO(model_path)

results = model.track(
    source=video_path,
    tracker="botsort.yaml",   # using BoT-SORT built-in
    persist=True,
    stream=True,              # memory safe
    conf=0.2,
    save=True,
    show=False
)


In [36]:
import cv2
import os

avi_files = [f for f in os.listdir("runs/detect/track") if f.endswith(".avi")]
if avi_files:
    input_file = os.path.join("runs/detect/track", avi_files[0])
    output_file = input_file.replace(".avi", "_converted.mp4")

    cap = cv2.VideoCapture(input_file)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_file, fourcc, cap.get(5),
                          (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()
    print(f"Converted to MP4: {output_file}")
else:
    print("No AVI file found.")


Converted to MP4: runs/detect/track/15sec_input_720p_converted.mp4


**Modify_botsort(Stride to avoid 0 player Detection Problem)**

In [37]:
from ultralytics import YOLO
import os

video_path = "15sec_input_720p.mp4"
model_path = "best.pt"
output_dir = "runs/detect/track"
os.makedirs(output_dir, exist_ok=True)

model = YOLO(model_path)

results = model.track(
    source=video_path,
    tracker="botsort.yaml",
    persist=True,
    stream=False,
    conf=0.3,          # 🔧 More strict detection
    iou=0.5,           # 🔧 Less overlap
    save=True,
    vid_stride=2       # 🔧 Skip alternate frames
)





WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/187) /content/15sec_input_720p.mp4: 384x640 16 players, 2 referees, 69.1ms
video 1/1 (frame 2/187) /content/15sec_input_720p.mp4: 384x640 12 players, 2 referees, 47.5ms
video 1/1 (frame 3/187) /content/15sec_input_720p.mp4: 384x640 13 players, 2 referees, 44.2ms
video 1/1 (frame 4/187) /content/15sec_input_720p.mp4: 384x640 14 players, 1 referee, 42.6ms
video 1/1 (frame 5/187) /content/15sec_input_720p.mp4: 384x640 15 players, 2 refe

In [38]:
import os

output_dir = "runs/detect/track3"
files = os.listdir(output_dir)
print(files)


['15sec_input_720p.avi']


In [39]:
!ffmpeg -i runs/detect/track3/15sec_input_720p.avi -vcodec libx264 runs/detect/track3/output_converted.mp4 -y


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [40]:
from google.colab import files
files.download('runs/detect/track3/output_converted.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
from ultralytics import YOLO
import os

video_path = "15sec_input_720p.mp4"
model_path = "best.pt"
output_dir = "runs/detect/track"
os.makedirs(output_dir, exist_ok=True)

model = YOLO(model_path)

results = model.track(
    source="15sec_input_720p.mp4",
    tracker="botsort.yaml",
    persist=True,
    stream=False,
    conf=0.25,         # Lower it if missing detections
    iou=0.5,           # Avoid multiple overlapping boxes
    save=True,
    vid_stride=2       # Helps with smoother ID assignment
)




WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/187) /content/15sec_input_720p.mp4: 384x640 16 players, 2 referees, 53.8ms
video 1/1 (frame 2/187) /content/15sec_input_720p.mp4: 384x640 12 players, 2 referees, 51.3ms
video 1/1 (frame 3/187) /content/15sec_input_720p.mp4: 384x640 13 players, 2 referees, 43.3ms
video 1/1 (frame 4/187) /content/15sec_input_720p.mp4: 384x640 14 players, 1 referee, 42.0ms
video 1/1 (frame 5/187) /content/15sec_input_720p.mp4: 384x640 15 players, 2 refe

In [42]:
!ffmpeg -i runs/detect/track4/15sec_input_720p.avi -vcodec libx264 runs/detect/track4/output_converted.mp4 -y

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [43]:
from google.colab import files
files.download("runs/detect/track4/output_converted.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**StrongSort(Not Completed)**

In [44]:
#StrongSORT + YOLOv8 repo (public + tested)
!git clone https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet.git
%cd Yolov5_StrongSORT_OSNet



Cloning into 'Yolov5_StrongSORT_OSNet'...
remote: Enumerating objects: 19914, done.
remote: Counting objects: 100% (562/562), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 19914 (delta 486), reused 449 (delta 449), pack-reused 19352 (from 3)
Receiving objects: 100% (19914/19914), 129.57 MiB | 31.04 MiB/s, done.
Resolving deltas: 100% (12919/12919), done.
/content/Yolov5_StrongSORT_OSNet


In [45]:
# Step 1: Install dependencies (ignore error if `requirements.txt` not found)
!pip install -r requirements.txt || true
!pip install lap cython_bbox gdown onnx onnxruntime ninja scikit-learn scikit-image filterpy


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.0 MB/s eta 0:00:00
  Created wheel for cython_bbox: filename=cython_bbox-0.1.5-cp311-cp311-linux_x86_64.whl size=106924 sha256=8df67ceeb7079946f762812afe4cfac3b305db7c03cfe3733d95daf81b99c90a
  Stored in directory: /root/.cache/pip/wheels/98/1b/e6/602b7896bbe63de2bd981e3c322026ffaeb1e70ddeb3f38974
Successfully built cython_bbox


In [46]:
# Make sure directory exists
!mkdir -p osnet_checkpoints

# Download working model file from GitHub release
!wget https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet/releases/download/osnet_checkpoints/osnet_x0_25_imagenet.pth -O osnet_checkpoints/osnet_x0_25_imagenet.pth


--2025-06-30 18:01:27--  https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet/releases/download/osnet_checkpoints/osnet_x0_25_imagenet.pth
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/mikel-brostrom/boxmot/releases/download/osnet_checkpoints/osnet_x0_25_imagenet.pth [following]
--2025-06-30 18:01:27--  https://github.com/mikel-brostrom/boxmot/releases/download/osnet_checkpoints/osnet_x0_25_imagenet.pth
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 404 Not Found
2025-06-30 18:01:27 ERROR 404: Not Found.



**DeepSort with Osnet(Not Completed)**

In [47]:
!git clone https://github.com/mikel-brostrom/Yolov5_DeepSort_Pytorch.git
%cd Yolov5_DeepSort_Pytorch

Cloning into 'Yolov5_DeepSort_Pytorch'...
remote: Enumerating objects: 19914, done.
remote: Counting objects: 100% (545/545), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 19914 (delta 468), reused 436 (delta 436), pack-reused 19369 (from 3)
Receiving objects: 100% (19914/19914), 129.51 MiB | 23.43 MiB/s, done.
Resolving deltas: 100% (12913/12913), done.
/content/Yolov5_StrongSORT_OSNet/Yolov5_DeepSort_Pytorch


In [48]:
!mkdir deep_sort/deep/checkpoint
!wget https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet/releases/download/osnet_0.1/osnet_x0_25_msmt17.pt -O deep_sort/deep/checkpoint/osnet_x0_25_msmt17.pt


mkdir: cannot create directory ‘deep_sort/deep/checkpoint’: No such file or directory
deep_sort/deep/checkpoint/osnet_x0_25_msmt17.pt: No such file or directory


In [49]:
!python track.py --yolo_model best.pt --source 15sec_input_720p.mp4 --img 640 --conf 0.2 --save-vid --classes 0


python3: can't open file '/content/Yolov5_StrongSORT_OSNet/Yolov5_DeepSort_Pytorch/track.py': [Errno 2] No such file or directory
